**The Bear Classifier!**

This classifier takes in bear images and determines the probability of the image being a grizzly bear, a black bear, or a teddy bear. Enjoy :)

In [6]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

In [10]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila 

You should consider upgrading via the 'C:\Users\miche\anaconda3\python.exe -m pip install --upgrade pip' command.


Enabling: voila
- Writing config: C:\Users\miche\anaconda3\etc\jupyter
    - Validating...
      voila 0.3.2 ok


In [11]:
key = os.environ.get('AZURE_SEARCH_KEY', '60f8c68cc3364a91bc2512ea1fa49aaf')
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [12]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [13]:
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink);

In [14]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))
dls = bears.dataloaders(path)
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
learn = cnn_learner(dls, resnet18, metrics=error_rate)


Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\miche/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [16]:
learn.fine_tune(4)

epoch,train_loss,valid_loss,error_rate,time
0,1.485714,0.453897,0.194444,00:45


epoch,train_loss,valid_loss,error_rate,time
0,0.304973,0.058799,0.013889,00:49
1,0.205078,0.027090,0.013889,00:47
2,0.151822,0.038989,0.013889,00:48
3,0.115204,0.044165,0.027778,00:48


In [17]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

In [18]:
path = Path()
learn_inf = load_learner(path/'export.pkl')
btn_upload = widgets.FileUpload()
btn_run = widgets.Button(description = "Classify")
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
out_pl = widgets.Output()
out_pl.clear_output()
btn_upload


FileUpload(value={}, description='Upload')

In [21]:
img = PILImage.create(btn_upload.data[-1])

with out_pl: display(img.to_thumb(128,128))
pred,pred_idx,probs = learn_inf.predict(img)
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])